In [ ]:
#@title Copyright 2019 Google LLC. { display-mode: "form" }
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table class="ee-notebook-buttons" align="left"><td>
<a target="_blank"  href="http://colab.research.google.com/github/google/earthengine-community/blob/master/guides/linked/ee-api-colab-setup.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run in Google Colab</a>
</td><td>
<a target="_blank"  href="https://github.com/google/earthengine-community/blob/master/guides/linked/ee-api-colab-setup.ipynb"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a></td></table>

# Earth Engine Python API Colab Setup

This notebook demonstrates how to setup the Earth Engine Python API in Colab and provides several examples of how to print and visualize Earth Engine processed data.

## Import API and get credentials

The Earth Engine API is installed by default in Google Colaboratory so requires only importing and authenticating. These steps must be completed for each new Colab session, if you restart your Colab kernel, or if your Colab virtual machine is recycled due to inactivity.

### Import the API

Run the following cell to import the API into your session.

In [ ]:
import ee

### Authenticate and initialize

Run the `ee.Authenticate` function to authenticate your access to Earth Engine servers and `ee.Initialize` to initialize it. Upon running the following cell you'll be asked to grant Earth Engine access to your Google account. Follow the instructions printed to the cell.

In [ ]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize(project='proimg-477121')

## Test the API

Test the API by printing the elevation of Mount Everest.

In [ ]:
# Print the elevation of Mount Everest.
dem = ee.Image('USGS/SRTMGL1_003')
xy = ee.Geometry.Point([86.9250, 27.9881])
elev = dem.sample(xy, 30).first().get('elevation').getInfo()
print('Mount Everest elevation (m):', elev)

## Map visualization

`ee.Image` objects can be displayed to notebook output cells. The following two
examples demonstrate displaying a static image and an interactive map.


### Static image

The `IPython.display` module contains the `Image` function, which can display
the results of a URL representing an image generated from a call to the Earth
Engine `getThumbUrl` function. The following cell will display a thumbnail
of the global elevation model.

In [ ]:
# Import the Image function from the IPython.display module.
from IPython.display import Image

# Display a thumbnail of global elevation.
Image(url = dem.updateMask(dem.gt(0))
  .getThumbURL({'min': 0, 'max': 4000, 'dimensions': 512,
                'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}))

### Interactive map

The [geemap](https://github.com/gee-community/geemap)
library can be used to display `ee.Image` objects on an interactive
[ipyleaflet](https://github.com/jupyter-widgets/ipyleaflet) map.

The following cell provides an example of using the `geemap.Map` object to
display an elevation model.

In [ ]:
# Import the geemap library.
import geemap

# Set visualization parameters.
vis_params = {
  'min': 0,
  'max': 4000,
  'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}

# Create a map object.
m = geemap.Map(center=[20, 0], zoom=3)

# Add the elevation model to the map object.
m.add_ee_layer(dem.updateMask(dem.gt(0)), vis_params, 'DEM')

# Display the map.
display(m)

## Chart visualization

Some Earth Engine functions produce tabular data that can be plotted by
data visualization packages such as `matplotlib`. The following example
demonstrates the display of tabular data from Earth Engine as a scatter
plot. See [Charting in Colaboratory](https://colab.sandbox.google.com/notebooks/charts.ipynb)
for more information.

In [ ]:
# Import the matplotlib.pyplot module.
import matplotlib.pyplot as plt

# Fetch a Landsat TOA image.
img = ee.Image('LANDSAT/LT05/C02/T1_TOA/LT05_034033_20000913')

# Select Red and NIR bands and sample 500 points.
samp_fc = img.select(['B3','B4']).sample(scale=30, numPixels=500)

# Arrange the sample as a list of lists.
samp_dict = samp_fc.reduceColumns(ee.Reducer.toList().repeat(2), ['B3', 'B4'])
samp_list = ee.List(samp_dict.get('list'))

# Save server-side ee.List as a client-side Python list.
samp_data = samp_list.getInfo()

# Display a scatter plot of Red-NIR sample pairs using matplotlib.
plt.scatter(samp_data[0], samp_data[1], alpha=0.2)
plt.xlabel('Red', fontsize=12)
plt.ylabel('NIR', fontsize=12)
plt.show()

In [ ]:
import ee
import geemap
import time

AOI = ee.Geometry.Polygon(
    [[[-47.50, -23.53],  # Longitude (X), Latitude (Y) - NO
      [-47.30, -23.53],  # Longitude (X), Latitude (Y) - NE
      [-47.30, -23.73],  # Longitude (X), Latitude (Y) - SE
      [-47.50, -23.73]]]) # Longitude (X), Latitude (Y) - SO

START_DATE = '2024-01-01'
END_DATE = '2025-01-01'
CLOUD_FILTER = 2

collection = ee.ImageCollection('COPERNICUS/S2_SR') \
    .filterBounds(AOI) \
    .filterDate(START_DATE, END_DATE) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER)) \
    .sort('CLOUDY_PIXEL_PERCENTAGE')

image_list = collection.limit(5).toList(5)

actual_size = image_list.size().getInfo()

print(f"INFO: Seus filtros encontraram {actual_size} imagens limpas.")

m = geemap.Map()
m.centerObject(AOI, 10)
m.add_basemap('HYBRID')

for i in range(actual_size):
    image = ee.Image(image_list.get(i))

    date = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd').getInfo()

    m.add_ee_layer(image.clip(AOI), vis_params_rgb, f'Cena {i+1} ({date})')

m.add_layer(AOI, {'color': 'yellow'}, 'Área de Interesse (AOI)')

display(m)

In [ ]:
import ee
import geemap

# ee.Authenticate()
# ee.Initialize()

AOI = ee.Geometry.Polygon(
    [[[128.7, -16.2],  # Longitude (X), Latitude (Y) - NO
      [128.9, -16.2],  # Longitude (X), Latitude (Y) - NE
      [128.9, -16.4],  # Longitude (X), Latitude (Y) - SE
      [128.7, -16.4]]]) # Longitude (X), Latitude (Y) - SO

START_DATE = '2022-01-01'
END_DATE = '2025-01-01'
CLOUD_FILTER = 10

def calculate_coverage(image):
    valid = image.select('B3').mask().gt(0)
    stats = valid.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=AOI,
        scale=100,
        maxPixels=1e9,
        bestEffort=True
    )
    coverage = ee.Number(stats.get('B3'))
    return image.set('COVERAGE', coverage)

collection = ee.ImageCollection('COPERNICUS/S2_SR') \
    .filterBounds(AOI) \
    .filterDate(START_DATE, END_DATE) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER)) \
    .sort('CLOUDY_PIXEL_PERCENTAGE') \
    .limit(20)

collection_with_coverage = collection.map(calculate_coverage)
best_image = collection_with_coverage \
    .filter(ee.Filter.gt('COVERAGE', 0.75)) \
    .sort('COVERAGE', False) \
    .first()

if best_image:
    image_to_export = best_image.select('B3', 'B8')
    date = ee.Date(best_image.get('system:time_start')).format('YYYYMMdd').getInfo()
    nome_da_tarefa = f'Lago_argyle_MELHOR_{date}'
    pasta_drive = 'Imagens_PI_Final'

    print(f"Iniciando exportação da melhor imagem ({nome_da_tarefa}.tif)...")

    geemap.ee_export_image_to_drive(
        image=image_to_export,
        description=nome_da_tarefa,
        folder=pasta_drive,
        scale=10,
        region=AOI.bounds()
    )

    print("Exportação concluída com sucesso!")
else:
    print("Nenhuma imagem com cobertura suficiente foi encontrada.")


In [ ]:
import ee
import geemap
import os

# --- Autenticação (descomente se for a 1ª vez) ---
# ee.Authenticate()
# ee.Initialize()

# --- PARÂMETROS DE ENTRADA ---
MAPBIOMAS_ASSET_ID = 'projects/mapbiomas-public/assets/brazil/lulc/collection10/mapbiomas_brazil_collection10_integration_v2'
CLASSE_AGUA = 33  # Classe “Água” no MapBiomas

# Região de interesse (exemplo: Represa de Guarapiranga)
AOI = ee.Geometry.Polygon(
    [[[-46.85, -23.60],
      [-46.65, -23.60],
      [-46.65, -23.80],
      [-46.85, -23.80]]])

NOME_LOCAL = 'Guarapiranga'

START_DATE = '2022-01-01'
END_DATE = '2023-12-31'
CLOUD_FILTER = 10
MIN_COVERAGE = 0.98

def calculate_coverage(image):
    valid_mask = image.select('B3').mask().gt(0)
    stats = valid_mask.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=AOI,
        scale=100,
        maxPixels=1e9
    )
    return image.set('COVERAGE', stats.get('B3'))

print(f" Analisando imagens Sentinel-2 para {NOME_LOCAL}...")

collection = (
    ee.ImageCollection('COPERNICUS/S2_SR')
    .filterBounds(AOI)
    .filterDate(START_DATE, END_DATE)
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER))
    .sort('CLOUDY_PIXEL_PERCENTAGE')
    .limit(50)
)

collection_with_coverage = collection.map(calculate_coverage)
good_collection = collection_with_coverage.filter(ee.Filter.gt('COVERAGE', MIN_COVERAGE))
best_image = good_collection.sort('CLOUDY_PIXEL_PERCENTAGE', True).first()

if best_image:
    date = ee.Date(best_image.get('system:time_start'))
    ano = date.get('year').getInfo()
    data_str = date.format('YYYY-MM-dd').getInfo()
    print(f" Melhor imagem encontrada: {data_str}")

    mapbiomas = ee.Image(MAPBIOMAS_ASSET_ID)
    banda_ano = f'classification_{ano}'
    gabarito = mapbiomas.select(banda_ano).eq(CLASSE_AGUA)

    pasta_img = 'Imagens_PI_Final'
    pasta_gab = 'Imagens_Ground_Truth'

    nome_img = f'{NOME_LOCAL}_IMAGEM_{ano}'
    print(f" Exportando {nome_img}.tif para '{pasta_img}'...")
    geemap.ee_export_image_to_drive(
        image=best_image.select('B3', 'B8'),
        description=nome_img,
        folder=pasta_img,
        scale=10,
        region=AOI.bounds()
    )

    nome_gab = f'{NOME_LOCAL}_GABARITO_{ano}'
    print(f" Exportando {nome_gab}.tif para '{pasta_gab}'...")
    geemap.ee_export_image_to_drive(
        image=gabarito,
        description=nome_gab,
        folder=pasta_gab,
        scale=30,
        region=AOI.bounds()
    )

    print(f"\n Exportações concluídas para {NOME_LOCAL}!")
    print("Os arquivos estarão disponíveis no Google Drive em alguns minutos.")
else:
    print("\n Nenhuma imagem com cobertura suficiente foi encontrada.")
